Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

do this first
in the anaconda prompt  or in the Google Colab

pip install pandas-datareader
pip install yfinance

In [171]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np 
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [172]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".
start = dt.datetime(2020,9,22)
end = dt.datetime.now()

stock_symbol = ['AVAX-USD']


In [173]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.
deletd_column =[]
df = pdr.get_data_yahoo(stock_symbol, start, end).dropna()
df
scaler = StandardScaler()

scaled_df = pd.DataFrame(
    scaler.fit_transform(df), 
    columns=df.columns,
    index=df.index)
#df1 = df[["Open","Adj Close"]]

#df = #TODO
#df.head()
#df.shape
#df1.columns
#df["Open"]
scaled_df= scaled_df.drop("Close",axis="columns")
#df1=df
#df1 = df1[["Open","Adj Close"]]
scaled_df


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Adj Close,Volume
Date,,,,,
2020-09-22,-0.950278,-0.682124,-0.969616,-0.938155,-0.180150
2020-09-23,-0.938415,-0.956543,-1.003936,-1.019423,-0.481873
2020-09-24,-1.020538,-0.979045,-1.003515,-0.969293,-0.532531
2020-09-25,-0.969546,-0.973064,-0.964042,-0.962978,-0.607559
2020-09-26,-0.962813,-0.980190,-0.956562,-0.971057,-0.661545
...,...,...,...,...,...
2021-11-05,2.601744,2.438793,2.486105,2.341856,0.383633
2021-11-06,2.373884,2.331678,2.489192,2.546849,0.187726
2021-11-07,2.590152,2.786905,2.773386,2.974831,1.728079


In [174]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

scaled_df['Close_15_Rolling'] = scaled_df['Adj Close'].rolling(16).mean() #start from zero ?

scaled_df['Open_1_Change'] = scaled_df['Open'].diff()

scaled_df['Adj Close_Change'] = scaled_df['Adj Close'].diff(-1)

scaled_df['dif_high_low']= scaled_df['High']-scaled_df['Low']

scaled_df['Volume_Change'] = scaled_df['Volume'].diff()



scaled_df=scaled_df.dropna()
X=scaled_df
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,Adj Close_Change,dif_high_low,Volume_Change
Date,,,,,,,,,,
2020-10-11,-0.996528,-1.009633,-0.977431,-0.992185,-0.691055,-0.987564,-0.010529,-0.002571,-0.032201,-0.005697
2020-10-14,-0.974396,-0.994973,-0.974817,-0.989615,-0.693734,-0.990781,0.022132,0.001077,-0.020157,-0.002679
2020-10-15,-0.990269,-1.007164,-0.975133,-0.990692,-0.699284,-0.988985,-0.015873,0.004122,-0.032031,-0.005551
2020-10-16,-0.991471,-1.009771,-0.981671,-0.994814,-0.694129,-0.990580,-0.001202,-0.007379,-0.028101,0.005156
2020-10-17,-0.995755,-1.007950,-0.977135,-0.987435,-0.700919,-0.992109,-0.004284,0.003793,-0.030815,-0.006790
...,...,...,...,...,...,...,...,...,...,...
2021-11-04,2.435321,2.433907,2.570415,2.575697,1.392356,1.956247,0.374313,0.233841,-0.136508,0.053231
2021-11-05,2.601744,2.438793,2.486105,2.341856,0.383633,2.000150,0.166422,-0.204993,-0.047313,-1.008723
2021-11-06,2.373884,2.331678,2.489192,2.546849,0.187726,2.051676,-0.227859,-0.427982,-0.157514,-0.195907


In [175]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
#np.where(a < 5, a, 10*a)
y = X["Adj Close_Change"]
y = np.where(y<0,-1,1)
y.shape
X.shape
X=X.drop("Adj Close_Change",axis="columns")
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,dif_high_low,Volume_Change
Date,,,,,,,,,
2020-10-11,-0.996528,-1.009633,-0.977431,-0.992185,-0.691055,-0.987564,-0.010529,-0.032201,-0.005697
2020-10-14,-0.974396,-0.994973,-0.974817,-0.989615,-0.693734,-0.990781,0.022132,-0.020157,-0.002679
2020-10-15,-0.990269,-1.007164,-0.975133,-0.990692,-0.699284,-0.988985,-0.015873,-0.032031,-0.005551
2020-10-16,-0.991471,-1.009771,-0.981671,-0.994814,-0.694129,-0.990580,-0.001202,-0.028101,0.005156
2020-10-17,-0.995755,-1.007950,-0.977135,-0.987435,-0.700919,-0.992109,-0.004284,-0.030815,-0.006790
...,...,...,...,...,...,...,...,...,...
2021-11-04,2.435321,2.433907,2.570415,2.575697,1.392356,1.956247,0.374313,-0.136508,0.053231
2021-11-05,2.601744,2.438793,2.486105,2.341856,0.383633,2.000150,0.166422,-0.047313,-1.008723
2021-11-06,2.373884,2.331678,2.489192,2.546849,0.187726,2.051676,-0.227859,-0.157514,-0.195907


In [176]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

#y_data = X["Adj Close_Change"] 
#x_data = X.drop("Adj Close_Change", axis = 1)


#index = int(0.7*len(X))
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=0)

y_test.shape

(118,)

In [177]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.
logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)


#logistic = LogisticRegression()
#logistic = #TODO


In [178]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.19034205308677377]
1,High,[0.19807166132557003]
2,Low,[-0.04035829368907972]
3,Adj Close,[-0.21031382283922717]
4,Volume,[-0.10012092898494745]
5,Close_15_Rolling,[-0.02248684362401194]
6,Open_1_Change,[-0.0009306320323115153]
7,dif_high_low,[0.2384299550146479]
8,Volume_Change,[-0.1327264634131037]


In [179]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
predictions

array([-1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1,
       -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1])

In [180]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

confusion_matrix =metrics.confusion_matrix(y_test,predictions)
confusion_matrix



array([[53,  8],
       [50,  7]], dtype=int64)

In [181]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test,predictions)
accuracy


0.5084745762711864

In [182]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X, y, cv=5)
cross_val

array([0.53164557, 0.46835443, 0.48717949, 0.43589744, 0.3974359 ])